# <b>Extrair dados chamadas do CNPq</b>

In [1]:
## Confirmar localização do requirements.txt no repo local:
# import os
# os.listdir('../../../../requirements.txt')
# %pip install --user -r ../../../../requirements.txt

## Caso seja necessário passar por validação SSL:
#sudo dpkg -i google-chrome-stable_current_amd64.deb
# !pip install --user -r ../../../../requirements.txt --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org

## Para instalar pelo Terminal:
# sudo apt-get update
# sudo apt-get install libcurl4-openssl-dev
# sudo apt-get install graphviz graphviz-dev

## Para instalar pacotes faltantes pela IDE:
# %pip install pyqt5
# %pip install pycurl
# %pip install selenium
# %pip install pygraphviz
# %pip install webdriver_manager
# %pip install --upgrade fitz
# %pip install --upgrade tools
# %pip install --upgrade pymupdf
# %pip install --upgrade frontend
# %pip install --upgrade markupsafe
# %pip install --upgrade notebook jupyterlab ipywidgets

## Verificação de versão do chromedriver no terminal:
# !pip3 install --upgrade pip
# !/usr/local/bin/chromedriver --version
# !curl -s https://chromedriver.storage.googleapis.com/LATEST_RELEASE

In [2]:
from pandas import concat
from getpass import getpass
from IPython.display import clear_output
import os, sys, subprocess, platform, locale, chardet
import pandas as pd 

## Configurar exibição dos dataframes do pandas na tela
locale.setlocale(locale.LC_ALL, '')
pd.set_option('display.max_colwidth', 75)
pd.set_option('colheader_justify', 'left')
pd.set_option('display.max_rows', 500)

def get_base_repo():
    """Retorna o caminho absoluto quatro níveis acima do diretório atual."""
    current_directory = os.getcwd()
    # Construir o caminho para subir quatro níveis
    path_five_levels_up = os.path.join(current_directory, '../../../../')
    # Normalizar o caminho para o formato absoluto
    absolute_path = os.path.abspath(path_five_levels_up)
    return absolute_path

def check_update():
  """
  Verifica se há atualizações para o Chromedriver e, se necessário, as instala.
  Funciona em Linux e Windows.
  """
  # Caminho para o script de atualização
  script_path = './../../../../scripts/update_check_chromedriver.sh'

  # Detectar o sistema operacional
  if platform.system() == "Linux":
    # Comando para verificar as versões no Linux
    command = f"bash {script_path}"
  elif platform.system() == "Windows":
    # Comando para verificar as versões no Windows
    command = fr"powershell -Command & {script_path}"
  else:
    raise NotImplementedError(f"Sistema operacional '{platform.system()}' não suportado.")

  # Adicionar parâmetro '-c' no Windows
  if platform.system() == "Windows":
    command += " -c"

  # Executar o comando
  command = f"'{command}'"
  process = subprocess.run(command, shell=True, capture_output=True)

  # Verificar o código de retorno
  if process.returncode != 0:
    # Detectar a codificação da saída
    encoding = chardet.detect(process.stderr)['encoding']

    # Decodificar a mensagem de erro com a codificação detectada
    error_message = process.stderr.decode(encoding)

    # Exibir a mensagem de erro
    print(f"Erro ao executar o comando: {error_message}")
    return

  # Verificar se a atualização é necessária
  output = process.stdout.decode()

  if "Versões principais do Chromedriver e Google Chrome estão iguais" in output:
    print("As versões do Chromedriver e Google Chrome estão atualizadas. Continuando execução...")
  else:
    print("As versões precisam ser atualizadas.")

    # Se a atualização for necessária, solicitar a senha do sudo no Linux
    if platform.system() == "Linux":
      sudo_password = getpass("Por favor, insira sua senha sudo:")

      # Executar o script de atualização com a senha do sudo
      command = f"echo '{sudo_password}' | sudo -S bash {script_path}"
    elif platform.system() == "Windows":
      # No Windows, a senha do administrador já está disponível no contexto do processo
      command = fr"powershell -Command & {script_path} -update"

    process = subprocess.run(command, shell=True, capture_output=True)

    # Limpar a saída para segurança
    clear_output(wait=True)

    # Exibir resultados
    print(process.stdout.decode())

    # Detectar a codificação da saída
    encoding = chardet.detect(process.stderr)['encoding']

    # Decodificar a mensagem de erro com a codificação detectada
    error_message = process.stderr.decode(encoding)

    # Exibir a mensagem de erro
    print(f"Erro ao executar o comando: {error_message}")

check_update()

# Adicionar pastas locais ao sys.path para permitir importação de pacotes
base_repo_dir = get_base_repo()
folder_utils = os.path.join(base_repo_dir, 'utils')
folder_domain = os.path.join(base_repo_dir, 'source', 'domain')
folder_data_input = os.path.join(base_repo_dir, 'data', 'input')
folder_data_output = os.path.join(base_repo_dir, 'data', 'output')
sys.path.append(folder_utils)
sys.path.append(folder_domain)
print(f"Sistema operacional: {sys.platform}")
print(f"Caminho base do repositório: {base_repo_dir}")

Erro ao executar o comando: ''powershell' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.
'.' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.

Sistema operacional: win32
Caminho base do repositório: c:\Users\marcos.aires\gml_classifier-1


## Chamadas CNPq abertas

In [ ]:
# Importar a classe e instarciar o método
from funding_finder_cnpq import FundingFinderCNPq
finder = FundingFinderCNPq()

# URL da página de resultados dos editais em aberto
url_abertas = "http://memoria2.cnpq.br/web/guest/chamadas-publicas?p_p_id=resultadosportlet_WAR_resultadoscnpqportlet_INSTANCE_0ZaM&filtro=abertas/"

df_abertas = finder.mount_dfchamadas(url_abertas)
df_abertas

In [ ]:
df_chamadas_abertas = finder.mount_final(df_abertas.copy(),'Chamadas CNPq abertas')
df_chamadas_abertas.to_csv('../../../../data/csv/df_cnpq_chamadas_abertas.csv', index=False)
finder.mount_foment_report(df_chamadas_abertas,'relatorio_chamadas_cnpq_abertas.html')

In [ ]:
# df_chamadas_abertas['titulo'].value_counts()

In [ ]:
# df_chamadas_abertas

In [ ]:
# from pprint import pprint
# pprint([x for x in df_chamadas_abertas['detalhes']], width=110)

## Chamadas CNPq encerradas

In [ ]:
# URL da página de resultados de editais encerrados
url_encerradas = "http://memoria2.cnpq.br/web/guest/chamadas-publicas?p_p_id=resultadosportlet_WAR_resultadoscnpqportlet_INSTANCE_0ZaM&filtro=encerradas/"

df_encerradas = finder.mount_dfchamadas_encerradas(url_encerradas)

In [ ]:
df_chamadas_encerradas = finder.mount_final(df_encerradas.copy(),'Chamadas CNPq encerradas')
df_chamadas_encerradas.to_csv('../../../../data/csv/df_cnpq_chamadas_encerradas.csv', index=False)
finder.mount_foment_report(df_chamadas_encerradas,'relatorio_chamadas_cnpq_encerradas.html')

In [ ]:
df_chamadas_encerradas['titulo'].value_counts()

## Gerar relatório geral das Chamadas do CNPq

In [ ]:
df = pd.concat([df_chamadas_abertas, df_chamadas_encerradas])
finder.mount_foment_report(df,'relatorio_chamadas_cnpq.html')

In [ ]:
df['titulo'].value_counts()

In [ ]:
import numpy as np
import requests, tempfile, fitz
from bs4 import BeautifulSoup
from datetime import datetime

def extract_inscricao_data(div_inscricao):
    """
    Extrai a data de início e a data de término das inscrições a partir de um elemento `div` com classe `inscricao`.
    Args:
        div_inscricao: Elemento BeautifulSoup da div com classe `inscricao`.
    Returns:
        Dicionário com as chaves "data_inicio_inscricao" e "data_termino_inscricao".
    """
    texto_li = div_inscricao.find("li").text.strip() # Extrair texto da li
    data_inicio, data_termino = texto_li.split(" a ") # Dividir texto em data de início e data de término
    # Formatar datas
    try:
        data_inicio = datetime.strptime(data_inicio.strip(), "%d/%m/%Y").date()
    except Exception as e:
        print(e)
        data_inicio = np.NaN
    try:
        data_termino = datetime.strptime(data_termino.strip(), "%d/%m/%Y").date()
    except:
        data_termino = np.NaN
    return {"data_inicio_inscricao": data_inicio, 
            "data_termino_inscricao": data_termino}

def extract_data(div_content, div_bottom_content):
    """Extrai dados da 'div_content' e o link do PDF da 'div_bottom_content'.
    Args:
        div_content: Objeto BeautifulSoup da 'div' com classe 'content'.
        div_bottom_content: Objeto BeautifulSoup da 'div' com classe 'bottom-content'.
    Returns:
        Dicionário contendo os dados extraídos ou None se não forem encontrados.
    """

    titulo = div_content.find("h4").text.strip()
    descricao = div_content.find("p").text.strip()
    data_inscricao = extract_inscricao_data(div_content)
    if not data_inscricao:
        data_inscricao = {"data_inicio_inscricao": None,
                          "data_termino_inscricao": None}
    link_pdf_element = div_bottom_content.find("a", alt="Chamada", class_="btn")
    if link_pdf_element:
        link_pdf = link_pdf_element.get('href')
        return {
            "link": link_pdf,
            "titulo": titulo,
            "descricao": descricao,
            **data_inscricao
        }
    else:
        print(f"Erro: Link do PDF não encontrado em {titulo}")
        link_pdf = None

def extract_links_chamada(soup):
    """
    Extrai os links dos botões "Chamada" da página.
    Argumentos:
        soup (BeautifulSoup): Objeto BeautifulSoup da página HTML.
    Retorna:
        list: Lista de strings contendo os links dos botões "Chamada".
    """

    chamada_links = []

    # Seleciona os botões "Chamada" com base em classes e atributos específicos.
    buttons = soup.find_all(
        "button",
        class_=["btn-chamada", "chamada-button"],
        attrs={"data-type": "chamada"},
    )

    # Extrai o link de cada botão "Chamada".
    for button in buttons:
        link = button.get("href")
        if link:
            chamada_links.append(link)

    # Retorna a lista de links.
    return chamada_links

def ler_pdf_link_temp(url_pdf):
    """
    Função para ler um PDF, gerando um arquivo temporário, a partir de um link usando PyMuPDF e retornar o texto extraído.
    Argumentos:
        url_pdf (str): URL do arquivo PDF.
    Retorno:
        str: Texto extraído do PDF.
    """
    substituicoes = {
        "\n": " ",
        "  ": " ",
        "Ɵ": "ti",
        "ơ": "ti",
        "‒": "-",
        "hƩp": "http",
        "LaƩes": "Lattes",
    }
    try:
        response = requests.get(url_pdf)
        print(response.status_code)
        response.raise_for_status()  # Raise an exception if download fails
        with tempfile.NamedTemporaryFile(suffix='.pdf') as temp_pdf: 
            temp_pdf.write(response.content)  # Salva conteúdo do PDF 
            temp_pdf.seek(0)  # Volta o ponteiro para o início do arquivo
            # Extrair texto do PDF
            texto_completo = ""
            with fitz.open(temp_pdf.name) as pdf_documento:  # Use o nome do arquivo
                for pagina in pdf_documento.pages():
                    texto_pagina = pagina.get_text("text")
                    for caracter, substituto in substituicoes.items():
                        texto_pagina = texto_pagina.replace(caracter, substituto)
                    texto_completo += texto_pagina.strip()
                    print(texto_completo)
        return texto_completo

    except requests.exceptions.RequestException as error:
        print(f"Error downloading PDF: {error}")
        return None

In [ ]:
url = "http://memoria2.cnpq.br/web/guest/chamadas-publicas?p_p_id=resultadosportlet_WAR_resultadoscnpqportlet_INSTANCE_0ZaM&filtro=abertas/"

chamadas_data = []
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
# print(soup.prettify())

divs_content = soup.find_all("div", class_="content", tabindex="0")
divs_bottom_content = soup.find_all("div", class_="bottom-content")
if len(divs_content) != len(divs_bottom_content):
    print("Erro: Quantidades diferentes de divs 'content' e 'bottom-content'")
    exit()

for div_content, div_bottom_content in zip(divs_content, divs_bottom_content):
    chamada_data = extract_data(div_content, div_bottom_content)
    chamadas_data.append(chamada_data)

pd.DataFrame(chamadas_data)